# Cursors - Streaming Large Result Sets

This notebook demonstrates **cursor operations** in elastic-script for processing large ES|QL result sets efficiently.

## Features

- **DECLARE CURSOR FOR** - Declare a cursor with an ES|QL query
- **OPEN cursor** - Execute query and make results available
- **FETCH cursor INTO** - Retrieve next row(s)
- **CLOSE cursor** - Release resources
- **Cursor attributes** - `%NOTFOUND`, `%ROWCOUNT`

## 1. Basic Cursor Usage

In [ ]:
-- Basic cursor workflow: DECLARE, OPEN, FETCH, CLOSE
CREATE PROCEDURE basic_cursor()
BEGIN
    -- 1. Declare cursor with ES|QL query
    DECLARE log_cursor CURSOR FOR
        FROM logs-sample
        | WHERE level = 'ERROR'
        | LIMIT 5;
    
    DECLARE row DOCUMENT;
    
    -- 2. Open cursor (executes the query)
    OPEN log_cursor;
    
    -- 3. Fetch rows one at a time
    FETCH log_cursor INTO row;
    
    WHILE NOT log_cursor%NOTFOUND LOOP
        PRINT 'Error: ' || row['message'];
        FETCH log_cursor INTO row;
    END LOOP;
    
    PRINT 'Total rows fetched: ' || log_cursor%ROWCOUNT;
    
    -- 4. Close cursor
    CLOSE log_cursor;
END PROCEDURE;

CALL basic_cursor();

## 2. Batch Fetching

In [ ]:
-- Fetch multiple rows at once with LIMIT
CREATE PROCEDURE batch_fetch()
BEGIN
    DECLARE log_cursor CURSOR FOR
        FROM logs-sample
        | LIMIT 20;
    
    DECLARE batch ARRAY;
    DECLARE batch_num NUMBER = 0;
    
    OPEN log_cursor;
    
    -- Fetch 5 rows at a time
    FETCH log_cursor LIMIT 5 INTO batch;
    
    WHILE ARRAY_LENGTH(batch) > 0 LOOP
        SET batch_num = batch_num + 1;
        PRINT 'Batch ' || batch_num || ': ' || ARRAY_LENGTH(batch) || ' rows';
        
        -- Process batch
        DECLARE i NUMBER;
        FOR i IN 1..ARRAY_LENGTH(batch) LOOP
            PRINT '  - ' || batch[i]['level'] || ': ' || SUBSTR(batch[i]['message'], 1, 40);
        END LOOP;
        
        -- Fetch next batch
        FETCH log_cursor LIMIT 5 INTO batch;
    END LOOP;
    
    PRINT 'Total processed: ' || log_cursor%ROWCOUNT;
    CLOSE log_cursor;
END PROCEDURE;

CALL batch_fetch();

## 3. Cursor Attributes

In [ ]:
-- Demonstrate cursor%NOTFOUND and cursor%ROWCOUNT
CREATE PROCEDURE cursor_attributes()
BEGIN
    DECLARE my_cursor CURSOR FOR
        FROM logs-sample
        | LIMIT 3;
    
    DECLARE row DOCUMENT;
    
    OPEN my_cursor;
    
    -- Before any fetch
    PRINT 'After OPEN:';
    PRINT '  %ROWCOUNT = ' || my_cursor%ROWCOUNT;
    PRINT '  %NOTFOUND = ' || my_cursor%NOTFOUND;
    
    -- First fetch
    FETCH my_cursor INTO row;
    PRINT 'After 1st FETCH:';
    PRINT '  %ROWCOUNT = ' || my_cursor%ROWCOUNT;
    PRINT '  %NOTFOUND = ' || my_cursor%NOTFOUND;
    
    -- Second fetch
    FETCH my_cursor INTO row;
    PRINT 'After 2nd FETCH:';
    PRINT '  %ROWCOUNT = ' || my_cursor%ROWCOUNT;
    
    -- Third fetch
    FETCH my_cursor INTO row;
    PRINT 'After 3rd FETCH:';
    PRINT '  %ROWCOUNT = ' || my_cursor%ROWCOUNT;
    
    -- Fourth fetch (past end)
    FETCH my_cursor INTO row;
    PRINT 'After 4th FETCH (past end):';
    PRINT '  %ROWCOUNT = ' || my_cursor%ROWCOUNT;
    PRINT '  %NOTFOUND = ' || my_cursor%NOTFOUND;
    
    CLOSE my_cursor;
END PROCEDURE;

CALL cursor_attributes();

## 4. Processing Large Datasets

In [ ]:
-- Process a large result set in batches with progress tracking
CREATE PROCEDURE process_large_dataset(batch_size NUMBER)
BEGIN
    DECLARE data_cursor CURSOR FOR
        FROM logs-sample
        | SORT @timestamp DESC
        | LIMIT 50;
    
    DECLARE batch ARRAY;
    DECLARE error_count NUMBER = 0;
    DECLARE warn_count NUMBER = 0;
    DECLARE info_count NUMBER = 0;
    
    OPEN data_cursor;
    PRINT 'Starting batch processing (batch size: ' || batch_size || ')...';
    
    FETCH data_cursor LIMIT batch_size INTO batch;
    
    WHILE ARRAY_LENGTH(batch) > 0 LOOP
        -- Process each row in the batch
        DECLARE i NUMBER;
        FOR i IN 1..ARRAY_LENGTH(batch) LOOP
            DECLARE row DOCUMENT = batch[i];
            
            IF row['level'] = 'ERROR' THEN
                SET error_count = error_count + 1;
            ELSIF row['level'] = 'WARN' THEN
                SET warn_count = warn_count + 1;
            ELSE
                SET info_count = info_count + 1;
            END IF;
        END LOOP;
        
        PRINT 'Processed ' || data_cursor%ROWCOUNT || ' rows so far...';
        
        -- Fetch next batch
        FETCH data_cursor LIMIT batch_size INTO batch;
    END LOOP;
    
    PRINT '--- Summary ---';
    PRINT 'Total processed: ' || data_cursor%ROWCOUNT;
    PRINT 'Errors: ' || error_count;
    PRINT 'Warnings: ' || warn_count;
    PRINT 'Info: ' || info_count;
    
    CLOSE data_cursor;
END PROCEDURE;

CALL process_large_dataset(10);

## 5. Error Handling with Cursors

In [ ]:
-- Use TRY/CATCH with cursors for safe cleanup
CREATE PROCEDURE safe_cursor_processing()
BEGIN
    DECLARE log_cursor CURSOR FOR
        FROM logs-sample
        | LIMIT 10;
    DECLARE row DOCUMENT;
    DECLARE processed NUMBER = 0;
    
    TRY
        OPEN log_cursor;
        FETCH log_cursor INTO row;
        
        WHILE NOT log_cursor%NOTFOUND LOOP
            -- Simulate potential error
            IF processed = 5 THEN
                THROW 'Simulated processing error' WITH CODE 'PROC_ERR';
            END IF;
            
            PRINT 'Processing row ' || processed;
            SET processed = processed + 1;
            
            FETCH log_cursor INTO row;
        END LOOP;
        
    CATCH
        PRINT 'Error occurred: ' || error['message'];
        PRINT 'Processed ' || processed || ' rows before error';
    FINALLY
        -- Always close the cursor
        CLOSE log_cursor;
        PRINT 'Cursor closed in FINALLY block';
    END TRY;
END PROCEDURE;

CALL safe_cursor_processing();

## 6. Cursor Lifecycle Reference

| Operation | Syntax | Description |
|-----------|--------|-------------|
| Declare | `DECLARE name CURSOR FOR query;` | Define cursor with ES\|QL query |
| Open | `OPEN name;` | Execute query, position before first row |
| Fetch One | `FETCH name INTO var;` | Get next row into DOCUMENT variable |
| Fetch Batch | `FETCH name LIMIT n INTO var;` | Get next n rows into ARRAY variable |
| Close | `CLOSE name;` | Release resources |

### Cursor Attributes

| Attribute | Type | Description |
|-----------|------|-------------|
| `cursor%NOTFOUND` | BOOLEAN | TRUE if last FETCH returned no rows |
| `cursor%ROWCOUNT` | NUMBER | Total rows fetched so far |